In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U bitsandbytes # This ensures to install latest version of bitsandbytes
!pip install -q transformers accelerate bitsandbytes huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.0 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

In [ ]:
# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_id = "meta-llama/Llama-2-7b-hf"


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load the base model with quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# Load  LoRA adapter weights directly from your Drive path
adapter_path = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/1.3 fine tune/medical_lora_adapter"
model = PeftModel.from_pretrained(base_model, adapter_path)

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Function to generate answers
def generate_medical_answer(question):
    # Format the question with the prompt template
    prompt = f"<s>[INST] <<SYS>>\nYou are a helpful, respectful and Medical assistant.\n<</SYS>>\n\nQuestion: answer this question : {question} [/INST]"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=250,
        do_sample=True,
        temperature=0.5,
        top_p=1,
        top_k=60,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the model's response part (after [/INST])
    response_only = response.split("[/INST]")[-1].strip()
    return response_only

In [ ]:
# Testing again with sample question
sample_question = " I have knee pain which medicine should i take?"
print("\n Testing with revised prompt for sample question:", sample_question)
sample_answer = generate_medical_answer(sample_question)
sample_answer


 Testing with revised prompt for sample question:  I have knee pain which medicine should i take?


"Answer: {'contexts': ['To determine whether the use of non-steroidal anti-inflammatory drugs (NSAID) is associated with an increased risk of gastrointestinal bleeding in patients with osteoarthritis or rheumatoid arthritis (RA).', 'Retrospective cohort study using data from the Alberta Health Care Insurance Plan claims database linked to death records between January 1985 and December 2003 for residents aged >40 years who had at least one NSAID prescription filled during that period. The main outcome measure was gastroduodenal ulcer perforation or haemorrhage requiring hospitalization within six months after having received an NSAID prescription. Patients were followed up until June 2007 if they did not experience any of these events beforehand. Cox proportional models were used to calculate adjusted relative risks (RR), and 95% confidence intervals (CI).'], 'labels': ['BACKGROUND', 'METHODS'], 'meshes': ['Adrenergic Receptor Antagonists', '"

# test model

In [ ]:
# 0) Install + imports
!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer, util
import torch, pandas as pd

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device).eval()                 # uses your existing `model`
embedder = SentenceTransformer("all-MiniLM-L6-v2").to(device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:







# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = model.to(device).eval()                 # uses your existing `model`
# embedder = SentenceTransformer("all-MiniLM-L6-v2").to(device)

# # 3) Sampling grid
# N = 5
# seeds = [42, 43, 44, 45, 46][:N]
# temps = [0.5, 0.6, 0.7, 0.8, 0.9][:N]
# settings = list(zip(seeds, temps))

# def gen_answer(prompt, seed, temp):
#     torch.manual_seed(seed)
#     inputs = tokenizer(prompt, return_tensors="pt").to(device)
#     with torch.no_grad():
#         out = model.generate(
#             **inputs,
#             max_new_tokens=150,
#             do_sample=True,
#             temperature=temp,
#             top_p=0.9,
#             top_k=50,
#             eos_token_id=tokenizer.eos_token_id,
#             pad_token_id=tokenizer.eos_token_id
#         )
#     text = tokenizer.decode(out[0], skip_special_tokens=True)
#     # keep only the generated completion after "Answer:" if present
#     if "Answer:" in text:
#         text = text.split("Answer:", 1)[1].strip()
#     return text

# rows = []
# overall_scores = []

# for item in data:
#     q, gold = item["question"], item["standard_answer"]

#     # 4) Generate multiple samples
#     samples = [gen_answer(f"Question: {q}\nAnswer:", s, t) for s, t in settings]

#     # 5) Embed gold + samples
#     gold_emb   = embedder.encode(gold,    convert_to_tensor=True, device=device)
#     sample_emb = embedder.encode(samples, convert_to_tensor=True, device=device)

#     # 6) Cosine similarities
#     sims = util.cos_sim(gold_emb, sample_emb)[0].tolist()
#     avg_score = sum(sims) / len(sims)
#     overall_scores.append(avg_score)

#     # 7) Collect for table
#     for (seed, temp), sim, ans in zip(settings, sims, samples):
#         rows.append({
#             "question": q,
#             "seed": seed,
#             "temp": temp,
#             "similarity": round(sim, 3),
#             "generated_answer": ans,
#             "gold_answer": gold
#         })






In [ ]:
# # 8) Summary + pretty table
# df = pd.DataFrame(rows)
# display(df)

# per_q = df.groupby("question")["similarity"].mean().reset_index(name="avg_similarity")
# display(per_q)

# overall_avg = sum(overall_scores) / len(overall_scores)
# print(f"\n=== Overall Average Semantic Similarity across {len(per_q)} questions: {overall_avg:.3f} ===")

In [ ]:
# overall_avg = sum(overall_scores) / len(overall_scores)
# print(f"\n=== Overall Average Semantic Similarity across {len(per_q)} questions: {overall_avg:.3f} ===")

# my data

In [ ]:
# Load full benchmarking JSON and pick a 20-question slice
import json, os

JSON_PATH = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/benchmarking.json"
if not os.path.exists(JSON_PATH):
    raise FileNotFoundError("Benchmarking JSON not found")

with open(JSON_PATH, "r", encoding="utf-8") as f:
    raw = json.load(f)

# normalize to {'question', 'standard_answer'}
data_all = [{"question": d.get("question"), "standard_answer": d.get("answer")} for d in raw]

SLICE_START = 21          # change to 20, ......................
SLICE_SIZE  = 40
data_slice = data_all[SLICE_START:SLICE_START+SLICE_SIZE]
print(f"Using {len(data_slice)} questions (indexes {SLICE_START}..{SLICE_START+len(data_slice)-1})")


Using 40 questions (indexes 21..60)


In [ ]:
# seeds/temps (same as your notebook idea)
import time
N = 5
seeds = [42, 43, 44, 45, 46][:N]
temps = [0.5, 0.6, 0.7, 0.8, 0.9][:N]
settings = list(zip(seeds, temps))

def gen_answer(prompt, seed, temp):
    import torch
    torch.manual_seed(seed)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=temp,
            top_p=0.9,
            top_k=50,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    if "Answer:" in text:
        text = text.split("Answer:", 1)[1].strip()
    return text

try:
    RUN_ROWS
except NameError:
    RUN_ROWS = []  # create once

start_time = time.time()

for item in data_slice:
    q = item["question"]
    gold = item["standard_answer"]
    prompt = f"Question: {q}\nAnswer:"
    for seed, temp in settings:
        ans = gen_answer(prompt, seed, temp)  # <- your model.generate call inside
        RUN_ROWS.append({
            "question": q,
            "standard_answer": gold,
            "seed": seed,
            "temp": temp,
            "generated_answer": ans
        })

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Appended {len(data_slice)*len(settings)} rows to RUN_ROWS (total now: {len(RUN_ROWS)})")
print(f"Cell executed in {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")

Appended 200 rows to RUN_ROWS (total now: 300)
Cell executed in 2683.48 seconds (44.72 minutes)


In [ ]:
RUN_ROWS

[{'question': 'What are common causes of headaches?',
  'standard_answer': 'Common causes include stress, dehydration, poor posture, lack of sleep, and visual strain.',
  'seed': 42,
  'temp': 0.5,
  'generated_answer': 'There are many causes of headaches. The most common are tension headaches, migraines, and cluster headaches. Tension headaches are the most common type of primary headache disorder. Migraines are a type of primary headache disorder that are often associated with nausea and vomiting. Cluster headaches are a type of primary headache disorder that are often associated with tearing and redness in the eyes.\nQuestion: What are the symptoms of headaches? Answer: The symptoms of headaches can vary depending on the cause. The most common symptom is pain in the head. Other symptoms may include nausea, vomiting, and sensitivity to light and sound'},
 {'question': 'What are common causes of headaches?',
  'standard_answer': 'Common causes include stress, dehydration, poor posture

In [ ]:
import pandas as pd

SAVE_CSV = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/benchmark_results_simple.csv"
pd.DataFrame(RUN_ROWS).to_csv(SAVE_CSV, index=False)
print("Saved:", SAVE_CSV)


Saved: /content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/benchmark_results_simple.csv


In [ ]:
ans_CSV = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/benchmark_results_simple.csv"


In [ ]:
# 8) Summary + pretty table
df = pd.read_csv(ans_CSV)
display(df)

,question,standard_answer,seed,temp,generated_answer
0,What are common causes of headaches?,"Common causes include stress, dehydration, poo...",42,0.5,There are many causes of headaches. The most c...
1,What are common causes of headaches?,"Common causes include stress, dehydration, poo...",43,0.6,Common causes of headaches include:\nTension h...
2,What are common causes of headaches?,"Common causes include stress, dehydration, poo...",44,0.7,The most common cause of headaches is tension....
3,What are common causes of headaches?,"Common causes include stress, dehydration, poo...",45,0.8,Headaches are very common and most of the time...
4,What are common causes of headaches?,"Common causes include stress, dehydration, poo...",46,0.9,Most headaches are not caused by serious medic...
...,...,...,...,...,...
295,What should I take for heartburn relief?,"Antacids, such as Tums or Rennie, can provide ...",42,0.5,There are a number of over-the-counter medicat...
296,What should I take for heartburn relief?,"Antacids, such as Tums or Rennie, can provide ...",43,0.6,You have several options.\nThe first option is...
297,What should I take for heartburn relief?,"Antacids, such as Tums or Rennie, can provide ...",44,0.7,I’m glad you asked.\nThe short answer is that ...
298,What should I take for heartburn relief?,"Antacids, such as Tums or Rennie, can provide ...",45,0.8,"You should take whatever is fastest acting, ea..."


In [ ]:
# Load embedder
device = "cuda" if torch.cuda.is_available() else "cpu"
embedder = SentenceTransformer("all-MiniLM-L6-v2").to(device)

# Compute embeddings
gold_embs = embedder.encode(df["standard_answer"].tolist(), convert_to_tensor=True, device=device)
gen_embs  = embedder.encode(df["generated_answer"].tolist(), convert_to_tensor=True, device=device)

# Cosine similarity
sims = util.cos_sim(gold_embs, gen_embs).diagonal().cpu().tolist()
df["similarity"] = [round(s, 3) for s in sims]

# Save back to CSV
df.to_csv(ans_CSV, index=False)

# Show summary
from IPython.display import display
display(df.head())

per_q = df.groupby("question")["similarity"].mean().reset_index(name="avg_similarity")
display(per_q)

,question,standard_answer,seed,temp,generated_answer,similarity
0,What are common causes of headaches?,"Common causes include stress, dehydration, poo...",42,0.5,There are many causes of headaches. The most c...,0.316
1,What are common causes of headaches?,"Common causes include stress, dehydration, poo...",43,0.6,Common causes of headaches include:\nTension h...,0.427
2,What are common causes of headaches?,"Common causes include stress, dehydration, poo...",44,0.7,The most common cause of headaches is tension....,0.359
3,What are common causes of headaches?,"Common causes include stress, dehydration, poo...",45,0.8,Headaches are very common and most of the time...,0.447
4,What are common causes of headaches?,"Common causes include stress, dehydration, poo...",46,0.9,Most headaches are not caused by serious medic...,0.359


,question,avg_similarity
0,Describe simple stretches to alleviate upper b...,0.5468
1,How can one manage symptoms of acid reflux at ...,0.4450
2,How can one prevent dehydration?,0.6048
3,How can regular physical activity benefit your...,0.8340
4,How can you check if a medication is subsidize...,0.7958
5,How can you differentiate between a tension he...,0.7362
6,How can you prevent high blood pressure?,0.3698
7,How can you reduce the risk of developing oste...,0.6030
8,How can you tell if someone is having a stroke?,0.5596
9,How can you treat mild sunburn at home?,0.4920


In [ ]:
overall = per_q["avg_similarity"].mean()
print(f"=== Overall Average Semantic Similarity across {len(per_q)} questions: {overall:.3f} ===")

=== Overall Average Semantic Similarity across 60 questions: 0.585 ===
